# Tests for Meeting on 13.11.2024

## Basic functionality

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from Bio import SeqIO

In [2]:
def load_fasta(filename):
    fasta_sequences = SeqIO.parse(open(filename),'fasta')
    seqs = []
    for fasta in fasta_sequences:
        name, sequence = fasta.id, str(fasta.seq)
        seqs.append(sequence)
    #print(len(sequence))
    for seq in seqs:
        print(len(seq))
    return seqs


def write_fasta(seqs, filename):
    with open(filename, "w") as file:
        for i in range(len(seqs)-1):
            line = parse_num_to_nuc(seqs[i])
            strline = "".join(line) + "\n"
            file.write("> " + str(i) + "\n" + strline)
        
        #print(line)
        file.write(">last\n")
        line = parse_num_to_nuc(seqs[len(seqs)-1])
        file.write("".join(line))


def get_reads_from(seq, read_len=200, coverage=5):
    L = len(seq)
    read_amt = int(L/read_len * coverage)
    reads = []
    for _ in range(read_amt):
        start = np.random.randint(0,L-read_len)
        read = seq[start:start+read_len]
        read = parse_nucleotides(read, caps=True)
        reads.append(read)
    return reads

def get_even_reads_from(seq, read_len=250, amt=6):
    step_size = int(len(seq)/(2*read_len))
    reads = []
    reads.append(seq)
    i=0
    while i <= len(seq)-read_len:
        reads.append(seq[i:i+read_len])
        randomness = np.random.randint(-2,2)
        i += int(read_len / amt) + randomness
    reads.append(seq[len(seq)-read_len:len(seq)])
    return reads


def parse_nucleotides(sequence, caps=False):
    new_seq = []
    if caps:
        map_to_vals = {"A": 1, "C": 2, "G": 3, "T":4}
    else:
        map_to_vals = {"a": 1, "c": 2, "g": 3, "t":4}
    for symbol in sequence:
        new_seq.append(map_to_vals[symbol])
        
    return new_seq


## Profile correction algo

In [3]:

def get_maxcount(pos, seqs_kmers, spaced_kmer_profile, target):
    #target = seqs[seq_to_investigate]
    f = len(spaced_kmer_profile)
    counts_i = []
    indexes = []
    len_loop = min(f-1, pos)
    start = pos-len_loop
    end = pos+1
    for i in range(start, end):
        if spaced_kmer_profile[pos-i] != 1:
            counts_i.append(0)
            indexes.append(-1)
            continue
        
        # Extract k mers starting at selected position
        spaced_kmer = target[i : i+f] * spaced_kmer_profile
        spaced_kmer = spaced_kmer[spaced_kmer != 0]
        s = ''.join(str(x) for x in spaced_kmer)
        counts_i.append(seqs_kmers[s])
        indexes.append(i)
    
    return max(counts_i), indexes[np.argmax(counts_i)]


def correct_counts(maxed_counts, maxed_count_indices, target_sequence, start_end_posis, seqs, diff_profile, kmer_profile):
    f = len(kmer_profile)
    correction_artifact = []
    last_correction = 0 
    # When corrections are necessary they are written into the maxed_counts array
    for i in range(1,len(maxed_count_indices)):
        # Make sure we have a change position on our hands.
        if maxed_counts[i] != maxed_counts[i-1]: 
            start_sum_at = last_correction
            end_sum_at = maxed_count_indices[i]+1
            
            if start_sum_at <= end_sum_at:
                correction = sum(start_end_posis[start_sum_at:end_sum_at])
            else:
                correction = -sum(start_end_posis[end_sum_at:start_sum_at])
                
            last_correction = maxed_count_indices[i] + 1
            diff_profile[i-1] += correction
            correction_artifact.append([correction, (start_sum_at, end_sum_at), start_end_posis[start_sum_at:end_sum_at]])

    return diff_profile, correction_artifact



def get_correction_profile(target, seqs, overlap_size, start_dict, end_dict):
    corr_profile = [0 for i in range(len(target)-overlap_size+2)]
    #corr_profile[0] = -1
    for i in range(len(target)-overlap_size):
        #
        #if i == 0:
            # Note that this whole check is necessary because our target sequence, i.e. the read we are investigating at the moment
            # also starts at the beginning and would thus be added to the count profile. We anticipate this by increasing the correction
            # profile at this point to one s.t. the loop below can reduce it to zero in the first step if just our sequence starts there.
            # If another sequence starts here, then the loop below will reduce the correction profile below zero
            #corr_profile[0] = 1
        start_snippet = ''.join(str(x) for x in target[i : i+overlap_size])      
        if start_snippet in start_dict:
            #print(target_snippet)
            #print(se_dict[target_snippet])
            corr_profile[i] -= start_dict[start_snippet]

    for i in range(overlap_size, len(target)+1):
        end_snippet = ''.join(str(x) for x in target[i-overlap_size : i])
        if end_snippet in end_dict:
            corr_profile[i-overlap_size+1] += end_dict[end_snippet]
    
        
    corr_profile[0]=0
    return corr_profile



def correct_diff_profile(filename, str_profile, seq_to_investigate, data=[]):
    seqs = []
    if data:
        for read in data:
            sequence_chars = [val for val in read]
            sequence = parse_nucleotides(sequence_chars)
            seqs.append(np.array(sequence))
    else:
        with open(filename) as file_in:
            for line in file_in:
                newline = line.rstrip('\n')
                sequence_chars = [char for char in newline]
                sequence = parse_nucleotides(sequence_chars)
                seqs.append(np.array(sequence))
    
    profile = [int(character) for character in str_profile]
    k = sum(profile)
    f = len(profile)
    
    # Turn into np arrays for componentwise multiplication
    profile = np.array(profile)
    
    # Count occurence of spaced k-mers
    starts = {}
    ends = {}
    seqs_kmers = {}
    for sequence in seqs:
        for i in range(len(sequence) - f):
            parsed_seq = parse_nucleotides(sequence)
            spaced_kmer = parsed_seq[i:i+f] * profile
            spaced_kmer = spaced_kmer[spaced_kmer != 0]
            s = ''.join(str(x) for x in spaced_kmer)
            if s not in seqs_kmers:
                seqs_kmers[s] = 1
            else:
                seqs_kmers[s] += 1

            if i == 0 or i == len(sequence)-f-1:
                solid_kmer = sequence[i:i+f]
                s2 = ''.join(str(x) for x in solid_kmer)
                addon = 0
                if i == 0:
                    starts[s2] = starts.get(s2, 0) + 1
                else: 
                    ends[s2] = ends.get(s2, 0) + 1
    
    # Get maxcounts from counts
    target = seqs[seq_to_investigate]
    xpoints = np.array([i for i in range(len(target) - f)])
    max_counts = []
    max_count_indices = []
    for i in range(len(xpoints)):
        maxp, argmaxp = get_maxcount(i, seqs, seqs_kmers, profile, seq_to_investigate=seq_to_investigate)
        max_counts.append(maxp)
        max_count_indices.append(argmaxp)
    #print(max_count_indices)
    # Get correction profile:
    start_end_posis = get_correction_profile(target, seqs, f, starts, ends)
    #print(start_end_posis)
    #print(start_end_posis)
    #print(max_count_indices)
    
    # Get diff profile:
    pre_corr_diff_profile = [max_counts[j] - max_counts[j-1] for j in range(1,len(max_counts))]
    
    # Apply correction strategy
    ypoints, correction_artifact = correct_counts(max_counts, max_count_indices, target, start_end_posis, seqs, pre_corr_diff_profile.copy(), profile)
    return xpoints[1:], ypoints, max_counts, start_end_posis, pre_corr_diff_profile, correction_artifact, max_count_indices


## Writing max count profiles to a file in a format that can be loaded into genome browser

In [20]:
long_seqs = load_fasta("../data/Pseudoalteromonas_translucida_KMM_520_genomic.fasta")
longer_chromosome = long_seqs[0]
shorter_chromosome = long_seqs[1]

3390388
757205


In [21]:
# Get reads from long sequence
seqs = get_reads_from(shorter_chromosome, read_len=250, coverage=10)
print(len(seqs))
seqs.insert(0, parse_nucleotides(shorter_chromosome, caps=True))

30288


In [22]:
# Get max counts from reads
seq_to_investigate = 0
str_profile = "1111110110110101110101011101011111101111"
profile = [int(character) for character in str_profile]
k = sum(profile)
f = len(profile)

# Turn into np arrays for componentwise multiplication
profile = np.array(profile)

print("building hashmap")
starts, ends = {}, {}
seqs_kmers = {}
for (sequence, j) in zip(seqs, range(len(seqs))):
    if j %3000 == 0: print("seq " + str(j))
    for i in range(len(sequence) - f):
        #if i%500000 == 0:
            #print(i)
        #parsed_seq = parse_nucleotides(sequence, caps=True)
        spaced_kmer = sequence[i:i+f] * profile
        spaced_kmer = spaced_kmer[spaced_kmer != 0]
        s = ''.join(str(x) for x in spaced_kmer)
        seqs_kmers[s] = seqs_kmers.get(s, 0) + 1

        if i == 0 or i == len(sequence)-f-1:
            solid_kmer = sequence[i:i+f]
            s2 = ''.join(str(x) for x in solid_kmer)
            addon = 0
            if i == 0:
                starts[s2] = starts.get(s2, 0) + 1
            else: 
                ends[s2] = ends.get(s2, 0) + 1

print("Getting and fixing maxcounts now")
# Get maxcounts from counts
target = seqs[seq_to_investigate]
xpoints = np.array([i for i in range(len(target) - f)])
max_counts = []
max_count_indices = []
for i in range(len(xpoints)):
    maxp, argmaxp = get_maxcount(i, seqs_kmers, profile, target)
    max_counts.append(maxp)
    max_count_indices.append(argmaxp)

print("done")
# Get correction profile:
#start_end_posis = get_correction_profile(target, seqs, f, starts, ends)

# Get diff profile:
#pre_corr_diff_profile = [max_counts[j] - max_counts[j-1] for j in range(1,len(max_counts))]

# Apply correction strategy
#ypoints, correction_artifact = correct_counts(max_counts, max_count_indices, target, start_end_posis, seqs, pre_corr_diff_profile.copy(), profile)

building hashmap
seq 0
seq 3000
seq 6000
seq 9000
seq 12000
seq 15000
seq 18000
seq 21000
seq 24000
seq 27000
seq 30000
Getting and fixing maxcounts now
done


In [23]:
with open("./mcProfile_chromo_spaced_short.wig", "w") as file:
    file.write("track type=wiggle_0\n")
    file.write("variableStep  chrom=chr2\n")
    for i in range(len(max_counts)):
        strline = str(i) + " " + str(max_counts[i]) + "\n"
        file.write(strline)

In [24]:
with open("./mcProfile_fixed_chromo_spaced_short.wig", "w") as file:
    file.write("track type=wiggle_0\n")
    file.write("fixedStep chrom=chr2 start=1 step=1\n")
    for i in range(len(max_counts)):
        strline = str(max_counts[i]) + "\n"
        file.write(strline)